In [ ]:
QDRANT_KEY='key-goes-here'
QDRANT_URL='url-goes-here'
COHERE_KEY='key-goes-here'
QDRANT_CLUSTER='MedRAG'
OPENAI_KEY='api-key-goes-here'
# !pip install -qq langchain qdrant-client requests google-generativeai pypdf cohere openai langchain-cohere selenium

In [ ]:
from langchain_cohere import CohereEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Qdrant

loader = DirectoryLoader(newpath, glob="**/*.pdf", loader_cls=PyPDFLoader, use_multithreading=True)
# loader = PyPDFLoader(url)
docs = loader.load_and_split()
embeddings = CohereEmbeddings(model="embed-english-v3.0", cohere_api_key=COHERE_KEY)

client = QdrantClient(QDRANT_URL, port=6333, grpc_port=6333)
qdrant = Qdrant.from_documents(docs, embeddings, url=QDRANT_URL, collection_name=QDRANT_CLUSTER, api_key=QDRANT_KEY, force_recreate=True)

TypeError: expected str, bytes or os.PathLike object, not list

In [ ]:
query = "ACL reconstruction"
found_docs = qdrant.similarity_search(query, k=1)
found_docs

[Document(page_content='Massachusetts General Brigham Sports Medicine  \nRehabilitation Protocol for Anterior Cruciate Ligament ( ACL ) \nReconstruction  \nThis protocol is intended to guide clinicians through the post -operative course for ACL Reconstruction. This protocol is \ntime based (dependent on tissue healing) as well as criterion based. Specific intervention should be based on the needs \nof the individual and should consider exam findings and clinical decision making. The timeframes for expected outcomes \ncontained within this guideline may vary based on  surgeon’s preference, additional procedures performed, and/or \ncomplications. If a clinician requires assistance in the progression of a post -operative patient, they should consult with \nthe referring surgeon.  \n \nThe interventions included within this protoco l are not intended to be an inclusive list of exercises. Therapeutic \ninterventions should be included and modified based on the progress of the patient and un

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from qdrant_client import QdrantClient

query = 'How can an ACL be repaired or rec'

client = QdrantClient(url=QDRANT_URL, prefer_grpc=True, api_key=QDRANT_KEY)

qdrant = Qdrant(client=client, collection_name=QDRANT_CLUSTER, embedding_function=embeddings)
search_results = qdrant.similarity_search(query, k=2)
chain = load_qa_chain(OpenAI(openai_api_key=OPENAI_KEY,temperature=0.2), chain_type="stuff")
results = chain({"input_documents": search_results, "question": query}, return_only_outputs=True)



ValueError: Neither of embeddings or embedding_function is set

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain.document_loaders import TextLoader



query = "Experimental Results"
found_docs = qdrant.similarity_search(query, k=4)

from langchain.chains.question_answering import load_qa_chain
# from langchain.llms import Gemini
from langchain.prompts import PromptTemplate


template = """Given the following extracted parts of a long document and a question, create a final answer.
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
QUESTION:
{question}
=========
CONTEXT:
{context}
=========
FINAL ANSWER:"""
PROMPT = PromptTemplate(template=template, input_variables=["context", "question"])


chain = load_qa_chain(OpenAI(temperature=0,openai_api_key=OPENAI_KEY), chain_type="stuff", prompt=PROMPT)
query = "how does the paper try to achive the results?"
chain.run(input_documents=found_docs, question=query)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}